In [ ]:
%pip install "zenml[server]"
!zenml intergration install sklearn -y
%pip install pyparsing == 2.4.2

import IPython

NumExpr defaulting to 2 threads.
Usage: zenml [OPTIONS] COMMAND [ARGS]...
Try 'zenml --help' for help.

Error: No such command 'intergration'.
ERROR: Invalid requirement: '=='


In [ ]:
# IPython.Application.instance().kernel.do_shutdown(restart=True)

In [ ]:
NGROK_TOKEN = "2bsrgNJhg3zaqDhsXwv63lbPzFZ_6oEg9u6PqU1RtGYrSuKvg"

In [ ]:
from zenml.environment import Environment

if Environment.in_google_colab(): #colab setup

  !pip install pyngrok
  !ngrok authtoken {NGROK_TOKEN}

Authtoken saved to configuration file: /root/.config/ngrok/ngrok.yml


##ZENML setup

In [ ]:
!rm -rf .ren
!zenml init

NumExpr defaulting to 2 threads.
Found existing ZenML repository at path '/content'.
⠋ Initializing ZenML repository at /content.
⠋ Initializing ZenML repository at /content.



##Example ML Code


In [ ]:
import numpy as np
from sklearn.base import ClassifierMixin
from sklearn.svm import SVC
from sklearn.datasets import load_digits
from sklearn.model_selection import train_test_split

def train_test()-> None:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test= train_test_split(
      data,digits.target,test_size=0.2,shuffle=True
  )
  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  test_acc = model.score(X_test,y_test)
  print(f"Test accuracy : {test_acc}")

train_test()

Test accuracy : 0.9916666666666667


In [ ]:
from zenml import step
from typing_extensions import Annotated
import pandas as pd
from sklearn.datasets import load_digits
from typing import Tuple

@step
def importer()->Tuple[
    Annotated[np.ndarray,"X_train"],
    Annotated[np.ndarray,"X_test"],
    Annotated[np.ndarray,"y_train"],
    Annotated[np.ndarray,"y_test"],
]:
  digits = load_digits()
  data = digits.images.reshape((len(digits.images),-1))
  X_train,X_test,y_train,y_test= train_test_split(
      data,digits.target,test_size=0.2,shuffle=True
  )
  return X_train,X_test,y_train,y_test


@step
def svc_trainer(
    X_train: np.ndarray,
    y_train: np.ndarray,
) -> ClassifierMixin:
  model = SVC(gamma=0.001)
  model.fit(X_train,y_train)
  return model


@step
def evaluator(
    X_test: np.ndarray,
    y_test: np.ndarray,
    model : ClassifierMixin,
)->float:
  test_acc = model.score(X_test,y_test)
  print(f"Test accuracy : {test_acc}")
  return test_acc

In [ ]:
from zenml import pipeline

@pipeline
def digits_pipeline():
  X_train,X_test,y_train,y_test=importer()
  model = svc_trainer(X_train=X_train,y_train=y_train)
  evaluator(X_test=X_test,y_test=y_test,model=model)

##Running ZenML Pipelines

In [ ]:
digits_svc_pipeline = digits_pipeline()
# digits_svc_pipeline.run(unlisted=True)

Initiating a new run for the pipeline: digits_pipeline.
Reusing registered pipeline version: (version: 3).
Executing a new run.
Using user: default
Using stack: default
  artifact_store: default
  orchestrator: default
Using cached version of importer.
Step importer has started.
Using cached version of svc_trainer.
Step svc_trainer has started.
Using cached version of evaluator.
Step evaluator has started.
Pipeline run has finished in 0.547s.
You can visualize your pipeline runs in the ZenML Dashboard. In order to try it locally, please run zenml up.


In [ ]:
#to visulize in dashboard

from zenml.environment import Environment

def start_zenml_dashboard(port = 8237):
  if Environment.in_google_colab():
    from pyngrok import ngrok

    public_url = ngrok.connect(port)
    print(f"\x1b[31mIn Colab, use this URL instead: {public_url}!\x1b[0m")
    !zenml up --blocking --port {port}
  else :
    !zenml up --port {port}

start_zenml_dashboard()

INFO:pyngrok.ngrok:Opening tunnel named: http-8237-1ebc33be-c087-420f-8d82-87b1bd67e04f


Opening tunnel named: http-8237-1ebc33be-c087-420f-8d82-87b1bd67e04f


INFO:pyngrok.process.ngrok:t=2024-02-04T05:15:53+0000 lvl=info msg="no configuration paths supplied"


t=2024-02-04T05:15:53+0000 lvl=info msg="no configuration paths supplied"


INFO:pyngrok.process.ngrok:t=2024-02-04T05:15:53+0000 lvl=info msg="using configuration at default config path" path=/root/.config/ngrok/ngrok.yml


...

t=2024-02-04T05:15:53+0000 lvl=info msg="started tunnel" obj=tunnels name=http-8237-1ebc33be-c087-420f-8d82-87b1bd67e04f addr=http://localhost:8237 url=https://7ca3-34-85-180-123.ngrok-free.app


INFO:pyngrok.process.ngrok:t=2024-02-04T05:15:53+0000 lvl=info msg=end pg=/api/tunnels id=640e22f3615c1d33 status=201 dur=28.87669ms


t=2024-02-04T05:15:53+0000 lvl=info msg=end pg=/api/tunnels id=640e22f3615c1d33 status=201 dur=28.87669ms
NumExpr defaulting to 2 threads.
The local ZenML dashboard is about to deploy in a blocking process. You can connect to it using the 
'default' username and an empty password.
Deploying a local ZenML server with name 'local'.
Starting ZenML Server as blocking process... press CTRL+C once to stop it.
INFO:     Started server process [38505]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8237 (Press CTRL+C to quit)


INFO:pyngrok.process.ngrok:t=2024-02-04T05:22:47+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"


t=2024-02-04T05:22:47+0000 lvl=info msg="received stop request" obj=app stopReq="{err:<nil> restart:false}"
INFO:     Shutting down
INFO:     Finished server process [38505]
ERROR:    Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/starlette/routing.py", line 686, in lifespan
    await receive()
  File "/usr/local/lib/python3.10/dist-packages/uvicorn/lifespan/on.py", line 137, in receive
    return await self.receive_queue.get()
  File "/usr/lib/python3.10/asyncio/queues.py", line 159, in get
    await getter
asyncio.exceptions.CancelledError



In [ ]:
!zenml up

NumExpr defaulting to 2 threads.
Deploying a local ZenML server with name 'local'.
⠋ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠙ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠹ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠸ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠼ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠴ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠦ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠧ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠇ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠏ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠋ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠙ Starting service 'LocalZenServer (type: zen_server, flavor: local)'.
⠹ Starting service 'LocalZenServer (type: zen_server, flavor: loc